# Laboratorio de regresión - 5

|                |   |
:----------------|---|
| **Nombre**     | Pablo Josué Panécatl García  |
| **Fecha**      | 15/09/2025  |
| **Expediente** |  751773 |

## Validación

Hemos estado usando `train_test_split` en nuestros modelos anteriores.

¿Por qué?

Porque no queremos usar todos los datos enteros. No queremos usarlos porque podemos cometer overfitting y no generalizar

Si la muestra es un subset de la población y queremos generalizar sobre la población, ¿no sería mejor utilizar todos los datos al entrenar un modelo?

Sí sería mejor pero, precisamente porque solo son una muestra, no queremos cometer overfitting con algo que solo es una muestra. No podría modelar la población entera

El propósito de volver a muestrear dentro de nuestro dataset es tener una idea de qué tan buena podría ser la generalización de nuestro modelo. Imagina un dataset ya separado en dos mitades. Utilizas la primera mitad para entrenar el modelo y pruebas en la segunda mitad; la segunda mitad eran datos invisibles para el modelo al momento de entrenar. Esto nos lleva a tres escenario típicos:

1. Si el modelo hace buenas predicciones en la segunda mitad, significa que la primera mitad era "suficiente" para generalizar.
2. Si el modelo no hace buenas predicciones en la segunda mitad, pero sí en la primera mitad, podría ser que había información importante en la segunda mitad que debió haber sido tomada en cuenta al entrenar, o un problema de overfitting.
3. Si el modelo no hace buenas predicciones en la segunda mitad, y tampoco en la primera mitad, se tendrían que revisar los factores y/o el modelo seleccionado.

El caso ideal sería el 1, pero por estadística los errores y varianzas tienen como entrada el número de muestas, por lo que tenemos menos seguridad de nuestros resutados al usar menos muestras. Si vemos que el modelo generaliza bien podemos unir de nuevo el dataset y entrenar sobre el dataset completo.

En el caso 2 está el problema de que no podemos saber qué información es necesaria para el entrenamiento apropiado del modelo; esto nos lleva a pensar que debemos usar el dataset completo para entrenar, pero esto nos lleva al mismo problema de no saber si el modelo puede generalizar.

El problema sólo incrementa si se tienen hiperparámetros en el modelo (e.g. $\lambda$ en regularización).

## Leave-One-Out Cross Validation

Este método de validación es una colección de $n$ `train-test-split`. Teniendo un dataset de $n$ muestras, la lógica es:
1. Saca una muestra del dataset.
2. Entrena tu modelo con las $n-1$ muestras.
3. Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
4. Regresa la muestra al dataset.
5. Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento $n$ veces para $n$ muestras.
6. Calcula la media y desviación estándar de los métricos guardados.

Con los resultados del proceso de validación podemos saber qué tan bueno podría ser el modelo seleccionado con los datos (con/sin transformaciones).

### Ejercicio 1

Utiliza el dataset `Motor Trend Car Road Tests`. Elimina la columna `model` y entrena 32 modelos diferentes utilizando Leave-One-Out Cross Validation con target `mpg`. Utiliza MSE como métrico.

In [3]:
import pandas as pd
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
import numpy as np

## Modelo 1/32

In [4]:
datos_crudos = pd.read_excel("C:\\Users\\pablo\\OneDrive - ITESO\\Semestre 5\\Laboratorio de aprendizaje estadístico\\Motor Trend Car Road Tests.xlsx")
datos = datos_crudos.drop(columns=["model"])

In [ ]:
MSE_lista = []

for i in range(32):
    #Sacamos una fila de datos
    datos_train = datos.drop(index=i)
    datos_test = datos.iloc[[i]]
    #Definimos X e y
    X_train = datos_train.drop(columns=["mpg"])
    y_train = datos_train["mpg"].values
    X_test = datos_test.drop(columns=["mpg"])
    y_test = datos_test["mpg"].values
    #estandarizamos
    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_train_sm = sm.add_constant(X_train_scaled)
    #Hacemos la regresión con el training estandarizado
    modelo_mpg = sm.OLS(y_train, X_train_sm).fit()
    #Predecimos
    X_test_scaled = scaler.transform(X_test)
    X_test_sm = sm.add_constant(X_test_scaled, has_constant='add')

    y_pred = modelo_mpg.predict(X_test_sm)
    #Calculamos el MSE
    MSE = mean_squared_error(y_test, y_pred)
    MSE_lista.append(MSE)

MSE_lista
print("El MSE promedio es: ", np.mean(MSE_lista))

#La desviación estándar del MSE es:
print("La desviación estándar del MSE es: ", np.std(MSE_lista))


El MSE promedio es:  12.181558006901975
La desviación estándar del MSE es:  17.06739987188869


Interpreta.

La desviación estándar entre los MSE de cada modelo es altísima. O sea, entre cada modelo, varía muchísimo. No se puede generalizar con estos modelos.

## K-Folds Cross-Validation

El dataset `Motor Trend Car Road Tests` sólo tiene 32 muestras, y utilizar un modelo sencillo de regresión múltiple hace que usar LOOCV sea muy rápido. El dataset `California Housing` tiene $20640$ muestras para $9$ columnas, entonces realizar un ajuste sobre una transformación o sobre el modelo y luego calcular el impacto esperado podría tomar más tiempo.

La solución propuesta es dividir el dataset en *k* folds (partes iguales), ajustar en *k-1* folds y probar en el restante.

### Ejercicio 2
Utiliza el dataset `California Housing` y haz K-folds Cross Validation con 10 folds. Utiliza el MSE como métrico.

In [6]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print("Dataset Shape:", housing.data.shape, housing.target.shape)
print("Dataset Features:", housing.feature_names)
print("Dataset Target:", housing.target_names)
X = housing.data
y = housing.target

Dataset Shape: (20640, 8) (20640,)
Dataset Features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
Dataset Target: ['MedHouseVal']


In [11]:
X_df = pd.DataFrame(X, columns = housing.feature_names)
y_df = pd.DataFrame(y, columns = ["MedHouseVal"])

In [14]:
df_housing = pd.concat([X_df, y_df], axis=1)
df_housing.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,1.6812,25.0,4.192201,1.022284,1392.0,3.877437,36.06,-119.01,0.47700
1,2.5313,30.0,5.039384,1.193493,1565.0,2.679795,35.14,-119.46,0.45800
2,3.4801,52.0,3.977155,1.185877,1310.0,1.360332,37.80,-122.44,5.00001
3,5.7376,17.0,6.163636,1.020202,1705.0,3.444444,34.28,-118.72,2.18600
4,3.7250,34.0,5.492991,1.028037,1063.0,2.483645,36.62,-121.93,2.78000


In [15]:
#Mezclamos los datos para asegurar que el split sea aleatorio
df_housing = df_housing.sample(frac=1, random_state=42).reset_index(drop=True)
df_housing.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,4.5625,46.0,4.845138,1.027611,1872.0,2.247299,37.97,-122.59,2.752
1,4.5000,17.0,5.262517,1.012179,2415.0,3.267930,34.23,-119.19,2.343
2,5.2174,5.0,7.306957,1.078261,3962.0,3.445217,33.95,-117.21,1.555
3,2.3083,20.0,5.485777,1.262582,1072.0,2.345733,38.96,-122.63,0.602
4,6.0000,26.0,5.442667,0.781333,936.0,2.496000,34.15,-117.24,1.402


In [16]:
#Ahora sí sacamos X e y
X = df_housing.drop(columns=["MedHouseVal"]).values.reshape(-1,8)
y = df_housing["MedHouseVal"].values

In [20]:
MSE_lista = []
k = 10
n = len(y)
fold_size = n // k

for i in range(k):
    X_train = np.concatenate([X[:i*fold_size], X[(i+1)*fold_size:]])
    y_train = np.concatenate([y[:i*fold_size], y[(i+1)*fold_size:]])
    X_val = X[i*fold_size:(i+1)*fold_size]
    y_val = y[i*fold_size:(i+1)*fold_size]

    #Estandarizamos
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)

    # Entrenamos el modelo
    model = sm.OLS(y_train, sm.add_constant(X_train)).fit()

    # Hacemos predicciones y calculamos el MSE
    y_pred = model.predict(sm.add_constant(X_val))
    mse = mean_squared_error(y_val, y_pred)
    MSE_lista.append(mse)

# Calculamos el MSE promedio
mse_promedio = np.mean(MSE_lista)
desvest = np.std(MSE_lista)
print("La desviación estándar del MSE es: ", desvest)
print("El MSE promedio es: ", mse_promedio)

La desviación estándar del MSE es:  0.022266703148738846
El MSE promedio es:  0.5270659954870699


In [ ]:
#Haremos la regresión usando todos los datos a ver si su mse se parece al promedio que sacamos

Interpreta.

In [23]:
X_df = pd.DataFrame(X, columns = housing.feature_names)
y_df = pd.DataFrame(y, columns = ["MedHouseVal"])

X_df_sm = sm.add_constant(X_df)
modelo_full = sm.OLS(y, X_df_sm).fit()
y_full_pred = modelo_full.predict(X_df_sm)
mse_full = mean_squared_error(y, y_full_pred)
mse_full

0.5243209861846073

El MSE del modelo usando todos los datos tiene uno parecido al del promedio de los folds. Eso es muy bueno ya que sí generaliza correctamente Yeiiii

## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3